In [ ]:
import numpy as np
import pandas as pd
import numpy as np
import glob as gb
import os
import keras_tuner as KT
import seaborn as sns
import PIL
import tensorflow as tf
from tensorflow import keras
from keras import layers, models
from sklearn.utils import resample
from keras.losses import BinaryCrossentropy
from sklearn.ensemble import AdaBoostClassifier
from scikeras.wrappers import KerasClassifier
from sklearn.ensemble import BaggingClassifier
import copy
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam
from sklearn.ensemble import AdaBoostClassifier, BaggingClassifier
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
from sklearn.model_selection import train_test_split
import shutil
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.utils import shuffle
# Enable GPU
os.environ["CUDA_VISIBLE_DEVICES"] = "0"


In [ ]:
train_df = pd.read_csv('archive/train.csv',sep=" ",header=None)
train_df.columns=['patient id', 'filename', 'label','data source']
train_df=train_df.drop(['patient id', 'data source'], axis=1 )
train_df.head()

In [ ]:
test_df = pd.read_csv('archive/test.csv',sep=" ",header=None)
test_df.columns=['patient id', 'filename', 'label','data source']
test_df=test_df.drop(['patient id', 'data source'], axis=1 )
test_df.head()

In [ ]:
train_p_count = train_df['label'].value_counts()['positive']
train_n_count = train_df['label'].value_counts()['negative']

print(train_df['label'].value_counts())
print(test_df['label'].value_counts())
train_df = shuffle(train_df)


In [ ]:
train_folder_path = 'archive/train'
test_folder_path = 'archive/test'

#Print out each folder's info

def count_photos(folder_path):
    count = 0
    for filename in os.listdir(folder_path):
        if filename.endswith(".jpg") or filename.endswith(".jpeg") or filename.endswith(".png"):
            count +=1
    return count


def count_photos_in_folders(train_folder_path, test_folder_path):
    train_count = count_photos(train_folder_path)
    test_count = count_photos(test_folder_path)
    return (train_count, test_count)

# 将数据信息打印出来
# 2. Print out info for each dataset
train_count, test_count = count_photos_in_folders(train_folder_path, test_folder_path)

print(f"Train folder contains {train_count} photos.")
print(f"Test folder contains {test_count} photos.")

train_data_size = [train_p_count, train_n_count]
labels = ['positive', 'negative']

fig, axs = plt.subplots(figsize=(12, 8))
sns.barplot(x=train_data_size, y=labels, ax=axs)
axs.set_title('Training data categories')
axs.set(xlabel='Image number')
plt.show()




In [ ]:
negative  = train_df[train_df['label']=='negative']   #negative values in class column
positive = train_df[train_df['label']=='positive']  #positive values in class column

df_majority_downsampled_neg = resample(negative, replace = True, n_samples = 5000)
df_majority_downsampled_pos = resample(positive, replace = True, n_samples = 5000)

train_df = pd.concat([df_majority_downsampled_pos, df_majority_downsampled_neg])
train_df = shuffle(train_df) # shuffling so that there is particular sequence

train_df, valid_df = train_test_split(train_df, train_size=0.8, random_state=0)

print(f"Negative and positive values of train:\n {train_df['label'].value_counts()}")
print(f"Negative and positive values of validation:\n {valid_df['label'].value_counts()}")
print(f"Negative and positive values of test:\n {test_df['label'].value_counts()}")

In [ ]:
# 读取数据，把train分成train set和validation set，将batch size设置为64

train_datagen = ImageDataGenerator(rescale = 1./255.,rotation_range = 40, width_shift_range = 0.2, height_shift_range = 0.2, 
                                   shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True, vertical_flip =True)
test_datagen = ImageDataGenerator(rescale = 1.0/255.)

#Now fit the them to get the images from directory (name of the images are given in dataframe) with augmentation


train_gen = train_datagen.flow_from_dataframe(dataframe = train_df, directory='archive/train', x_col='filename', 
                                              y_col='label', target_size=(256,256), batch_size=64, 
                                               class_mode='binary')
val_gen = test_datagen.flow_from_dataframe(dataframe = valid_df, directory='archive/train', x_col='filename',
                                             y_col='label', target_size=(256,256), batch_size=64, 
                                            class_mode='binary')
test_gen = test_datagen.flow_from_dataframe(dataframe = test_df, directory='archive/test', x_col='filename', 
                                            y_col='label', target_size=(256,256), batch_size=64,
                                             class_mode='binary')
labels = {value: key for key, value in train_gen.class_indices.items()}
#class mode binary because we want the classifier to predict covid or not
#target size (200,200) means we want the images to resized to 200*200

#Examine the first image in the training dataset and print the label which corresponding to it.
print(labels)
images, labels = next(train_gen)
# Get the filenames associated with the images in the batch
filenames = train_gen.filenames
# Print the filename and label for the first image in the batch
print(filenames[0], labels[0])
# Plot the first image in the batch
plt.imshow(images[0])

In [ ]:
from keras.models import Model

# Load your fine-tuned CNN model
cnn_model = keras.models.load_model('Best_CNN_only')

# Remove the last layer to use the model as a feature extractor
feature_extractor = Model(inputs=cnn_model.input, outputs=cnn_model.layers[-2].output)
train_features = feature_extractor.predict(train_gen)
val_features = feature_extractor.predict(val_gen)
test_features = feature_extractor.predict(test_gen)


train_labels = np.concatenate([train_gen[i][1] for i in range(len(train_gen))])
val_labels = np.concatenate([val_gen[i][1] for i in range(len(val_gen))])
test_labels = np.concatenate([test_gen[i][1] for i in range(len(test_gen))])


base_estimator = DecisionTreeClassifier(max_depth=2)
adaboost = AdaBoostClassifier(base_estimator=base_estimator, n_estimators=20)

adaboost.fit(train_features, train_labels)
val_preds = adaboost.predict(val_features)
val_accuracy = accuracy_score(val_labels, val_preds)

print(f"Validation accuracy: {val_accuracy}")

test_preds = adaboost.predict(test_features)
test_accuracy = accuracy_score(test_labels, test_preds)

print(f"Test accuracy: {test_accuracy}")
